In [8]:
from model import CardClassifier
from consts import LEARNING_RATE, BATCH_SIZE, SHUFFLE, EPOCHS
from data_loader import CardDataset
import torch.nn as nn
import torch.optim as optim
import json
from torch.utils.data import DataLoader
import torch
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

### Fetch the data

In [2]:
data_loader = CardDataset()

train_set = data_loader.train
test_set = data_loader.test
valid_set = data_loader.val

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE)
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=SHUFFLE)

Dataset downloaded to: /Users/josephkrueger/.cache/kagglehub/datasets/gpiosenka/cards-image-datasetclassification/versions/2


Class mappings

In [3]:
class_mapping = {v: k for k, v in train_set.class_to_idx.items()}
print(class_mapping)

# save to json
with open('class_mapping.json', 'w') as f:
    json.dump(class_mapping, f)

image, label = train_set[100]
image.shape

{0: 'ace of clubs', 1: 'ace of diamonds', 2: 'ace of hearts', 3: 'ace of spades', 4: 'eight of clubs', 5: 'eight of diamonds', 6: 'eight of hearts', 7: 'eight of spades', 8: 'five of clubs', 9: 'five of diamonds', 10: 'five of hearts', 11: 'five of spades', 12: 'four of clubs', 13: 'four of diamonds', 14: 'four of hearts', 15: 'four of spades', 16: 'jack of clubs', 17: 'jack of diamonds', 18: 'jack of hearts', 19: 'jack of spades', 20: 'joker', 21: 'king of clubs', 22: 'king of diamonds', 23: 'king of hearts', 24: 'king of spades', 25: 'nine of clubs', 26: 'nine of diamonds', 27: 'nine of hearts', 28: 'nine of spades', 29: 'queen of clubs', 30: 'queen of diamonds', 31: 'queen of hearts', 32: 'queen of spades', 33: 'seven of clubs', 34: 'seven of diamonds', 35: 'seven of hearts', 36: 'seven of spades', 37: 'six of clubs', 38: 'six of diamonds', 39: 'six of hearts', 40: 'six of spades', 41: 'ten of clubs', 42: 'ten of diamonds', 43: 'ten of hearts', 44: 'ten of spades', 45: 'three of clu

torch.Size([3, 128, 128])

### Create the model

In [4]:
model = CardClassifier(num_classes=len(class_mapping))
print(str(model))

CardClassifier(
  (base): EfficientNet(
    (conv_stem): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn1): BatchNormAct2d(
      32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
      (drop): Identity()
      (act): SiLU(inplace=True)
    )
    (blocks): Sequential(
      (0): Sequential(
        (0): DepthwiseSeparableConv(
          (conv_dw): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (bn1): BatchNormAct2d(
            32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True
            (drop): Identity()
            (act): SiLU(inplace=True)
          )
          (aa): Identity()
          (se): SqueezeExcite(
            (conv_reduce): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (act1): SiLU(inplace=True)
            (conv_expand): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (gate): Sigmoid()
          )
          (conv_pw): Co

In [5]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print("Device: ", device)

Device:  cpu


##### Loss: Cross-Entropy | Optimizer: Adam

In [6]:
print("Learning rate: ", LEARNING_RATE)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

Learning rate:  0.001


In [7]:
def perform_step(data, labels, mdl, opt, lf):
    opt.zero_grad() # clear gradient
    output = mdl(images) # forward pass
    loss = lf(output, labels) # calculate loss
    loss.backward() # backward pass
    opt.step() # update weights
    return loss

train_losses, val_losses = [], []

for epoch in range(EPOCHS):
    model.train()
    loss_total = 0.0
    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        perform_step(images, labels, model, optimizer, loss_function)
    
    # epoch loss
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)

    # validation 
    model.eval()
    loss_total = 0.0

    with torch.no_grad():
        for images, labels in tqdm(valid_loader, desc="Validation"):
            images, labels = images.to(device), labels.to(device)
            perform_step(images, labels, model, optimizer, loss_function)

    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    print(f"Epoch {epoch+1}/{num_epochs} - Train loss: {train_loss}, Validation loss: {val_loss}")


Training:   0%|          | 0/239 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.plot(train_losses, label='Training loss')
plt.plot(val_losses, label='Validation loss')
plt.legend()
plt.title("Loss over epochs")
plt.show()